In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import re
from scipy import stats
import time


In [73]:
def get_paper_links_scrape(posters=0):
    # paper_link_CSS = 'ul > li > h4 > a:nth-child(1)'
    session = webdriver.Chrome()
    session.implicitly_wait(10)
    session.get('https://openreview.net/group?id=ICLR.cc/2019/Conference')
    paper_links = set()
    if posters == 1:
        session.find_element_by_css_selector('#notes > div > ul > li:nth-child(2) > a').click()
    elif posters == 2:
        session.find_element_by_css_selector('#notes > div > ul > li:nth-child(3) > a').click()

    header = 'accepted-oral-papers'
    if posters == 1:
        header = 'accepted-poster-papers'
    elif posters==2:
        header = 'rejected-papers'
    paper_link_CSS = f'#{header} > ul > li > h4 > a:nth-child(1)'
    prev_len = []
    for i in range(100):
        session.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        new_papers = set(session.find_elements_by_css_selector(paper_link_CSS))
        new_papers = set([i.get_attribute('href') for i in new_papers])
        paper_links = paper_links.union(new_papers)
        time.sleep(1)
        if len(prev_len) > 5 and len(paper_links) == prev_len[-5]:
            break
        prev_len.append(len(paper_links))
    new_papers = set(session.find_elements_by_css_selector(paper_link_CSS))
    new_papers = set([i.get_attribute('href') for i in new_papers])
    paper_links = paper_links.union(new_papers)
    session.close()
    return paper_links

In [74]:
def get_paper_links_load(filename):
    return [i['url'] for i in json.load(open(filename))]


In [75]:
paper_links = get_paper_links_scrape(2) | get_paper_links_scrape(1) | get_paper_links_scrape(0)
# paper_links = get_paper_links_load('data/iclr2018.json')

In [76]:
paper_links = list(paper_links)
papers = {}
print(len(paper_links))

1579


In [77]:
session = webdriver.Chrome()
start_location = len(papers)

idx = start_location
while idx < len(paper_links):
    link = paper_links[idx]
    print(len(papers.items()), link)
    data = {}
 
    session.get(link)
    time.sleep(2)

    def update_non_comments():
        title = session.find_element_by_css_selector('div.title_pdf_row > h2 > a:nth-child(1)').get_attribute('text')
        abstract = session.find_element_by_css_selector('#content > div:nth-child(1) > div.note_contents > span.note_content_value').get_attribute('innerHTML')
        
        data['title'] = title
        data['abstract'] = abstract
        authors = session.find_element_by_css_selector('.signatures').find_elements_by_css_selector('a')
        data['authors'] = [i.get_attribute('innerText') for i in authors]
        data['emails'] = [i.get_attribute('data-original-title') for i in authors]
        data['emails'] = list(set([i[i.find('@')+1:] for i in data['emails']]))
        print(data['authors'], data['emails'])
        papers[link] = data
    
    comment_CSS = '#note_children > div.note_with_children'
    try:
        WebDriverWait(session, 30).until(
            lambda session: len(session.find_elements_by_css_selector(comment_CSS)) >=1)
    except:
        if len(session.find_elements_by_css_selector('.spinner')) == 1:
            continue
        elif session.current_url =='https://openreview.net/':
            continue
        elif '502' in session.title:
            continue
        elif '404' in session.title:
            idx += 1
            continue
        else:
            idx += 1
            update_non_comments()
            continue
    comments = session.find_elements_by_css_selector(comment_CSS)
    comment_vals = []
    for comment in comments:
        sections = comment.find_elements_by_css_selector('.note_contents')
        section_vals = {}
        for section in sections:
            field = section.find_element_by_css_selector('.note_content_field')
            field = field.get_attribute('innerHTML').strip()
            content = section.find_element_by_css_selector('.note_content_value')
            content = content.get_attribute('innerHTML').strip()
            section_vals[field] = content
        comment_vals.append(section_vals)
    update_non_comments()
    data['comments'] = comment_vals
    
    papers[link] = data 
    idx += 1

0 https://openreview.net/forum?id=HkzL4hR9Ym
['Olivier Tieleman', 'Angeliki Lazaridou', 'Shibl Mourad', 'Charles Blundell', 'Doina Precup'] ['google.com']
1 https://openreview.net/forum?id=ByGUFsAqYm
['Adityanarayanan Radhakrishnan', 'Caroline Uhler', 'Mikhail Belkin'] ['cse.ohio-state.edu', 'mit.edu']
2 https://openreview.net/forum?id=BJgsN3R9Km
['Samyam Rajbhandari', 'Harsh Shrivastava', 'Yuxiong He'] ['microsoft.com', 'gatech.edu']
3 https://openreview.net/forum?id=rylIy3R9K7
['Songtao Lu', 'Rahul Singh', 'Xiangyi Chen', 'Yongxin Chen', 'Mingyi Hong'] ['gatech.edu', 'umn.edu']
4 https://openreview.net/forum?id=S1eK3i09YQ
['Simon S. Du', 'Xiyu Zhai', 'Barnabas Poczos', 'Aarti Singh'] ['cs.cmu.edu', 'mit.edu', 'cmu.edu']
5 https://openreview.net/forum?id=r1gKNs0qYX
['Lei Gu'] ['uci.edu']
6 https://openreview.net/forum?id=rkMD73A5FX
['Michael Tsang', 'Youbang Sun', 'Dongxu Ren', 'Beibei Xin', 'Yan Liu'] ['usc.edu', 'mails.tsinghua.edu.cn', 'mail.ustc.edu.cn']
7 https://openreview.net/f

['Baoxiang Wang', 'Tongfang Sun', 'Xianjun Sam Zheng'] ['gmail.com', 'uw.edu', 'deephow.com']
58 https://openreview.net/forum?id=SJfFTjA5KQ
['Murphy Yuezhen Niu', 'Lior Horesh', "Michael O'Keeffe", 'Isaac Chuang'] ['ll.mit.edu', 'mit.edu', 'us.ibm.com']
59 https://openreview.net/forum?id=HylsgnCcFQ
['Aravind Sankar', 'Yanhong Wu', 'Liang Gou', 'Wei Zhang', 'Hao Yang'] ['visa.com', 'illinois.edu']
60 https://openreview.net/forum?id=B1GIQhCcYm
['Samuel Lavoie-Marchildon', 'Sebastien Lachapelle', 'Mikołaj Bińkowski', 'Aaron Courville', 'Yoshua Bengio', 'R Devon Hjelm'] ['gmail.com', 'microsoft.com', 'umontreal.ca']
61 https://openreview.net/forum?id=HyxCxhRcY7
['Dan Hendrycks', 'Mantas Mazeika', 'Thomas Dietterich'] ['oregonstate.edu', 'berkeley.edu', 'ttic.edu']
62 https://openreview.net/forum?id=Hygn2o0qKX
['Vaishnavh Nagarajan', 'Zico Kolter'] ['cs.cmu.edu']
63 https://openreview.net/forum?id=S1zz2i0cY7
['Johannes Ballé', 'Nick Johnston', 'David Minnen'] ['google.com']
64 https://openr

['Tianchen Zhao', 'Dejiao Zhang', 'Zeyu Sun', 'Honglak Lee'] ['eecs.umich.edu', 'umich.edu']
114 https://openreview.net/forum?id=rkeX-3Rqtm
['Lukas Nabergall', 'Justin Toth', 'Leah Cousins'] ['uwaterloo.ca']
115 https://openreview.net/forum?id=HyxKIiAqYQ
['Jooyoung Lee', 'Seunghyun Cho', 'Seung-Kwon Beack'] ['etri.re.kr']
116 https://openreview.net/forum?id=HyleYiC9FX
['Tolgahan Cakaloglu', 'Christian Szegedy', 'Xiaowei Xu'] ['ualr.edu', 'google.com']
117 https://openreview.net/forum?id=BJgTZ3C5FX
['Akihiro Iohara', 'Takahito Ogawa', 'Toshiyuki Tanaka'] ['datagrid.co.jp', 'sys.i.kyoto-u.ac.jp', 'i.kyoto-u.ac.jp']
118 https://openreview.net/forum?id=rkx0g3R5tX
['Ugo Tanielian', 'Flavian vasile', 'Mike Gartrell'] ['criteo.com']
119 https://openreview.net/forum?id=Skluy2RcK7
['Ella M. Gale', 'Anh Nguyen', 'Ryan Blything', 'Nicholas Martin and Jeffrey S. Bowers'] ['auburn.edu', 'gmail.com', 'bristol.ac.uk']
120 https://openreview.net/forum?id=ryG2Cs09Y7
['Chihuang Liu', 'Joseph JaJa'] ['um

['Giuseppe Marra', 'Dario Zanca', 'Alessandro Betti', 'Marco Gori'] ['unisi.it', 'unifi.it']
170 https://openreview.net/forum?id=HkNN7nR5Ym
['Song-Hao Jia', 'Ding-Jie Chen', 'Hwann-Tzong Chen'] ['gmail.com', 'icloud.com', 'cs.nthu.edu.tw']
171 https://openreview.net/forum?id=SkVhlh09tX
['Felix Wu', 'Angela Fan', 'Alexei Baevski', 'Yann Dauphin', 'Michael Auli'] ['dauphin.io', 'gmail.com', 'cornell.edu', 'fb.com']
172 https://openreview.net/forum?id=HkGsHj05tQ
['Zhaodong Chen', 'Lei Deng', 'Guoqi Li', 'Jiawei Sun', 'Xing Hu', 'Ling Liang', 'YufeiDing', 'Yuan Xie'] ['mails.tsinghua.edu.cn', 'mail.tsinghua.edu.cn', 'cs.ucsb.edu', 'ucsb.edu']
173 https://openreview.net/forum?id=S1GkToR5tm
['Samaneh Azadi', 'Catherine Olsson', 'Trevor Darrell', 'Ian Goodfellow', 'Augustus Odena'] ['eecs.berkeley.edu', 'berkeley.edu', 'google.com']
174 https://openreview.net/forum?id=HJMXTsCqYQ
['Ryan-Rhys Griffiths', 'José Miguel Hernández-Lobato'] ['cam.ac.uk']
175 https://openreview.net/forum?id=SklXvs0qt

['Chuanbiao Song', 'Kun He', 'Liwei Wang', 'John E. Hopcroft'] ['hust.edu.cn', 'pku.edu.cn', 'cs.cornell.edu']
225 https://openreview.net/forum?id=HJMsiiRctX
['Fahad Alhasoun'] ['mit.edu']
226 https://openreview.net/forum?id=B1xnPsA5KX
['Zhenwen Dai', 'Eric Meissner', 'Neil D. Lawrence'] ['amazon.com']
227 https://openreview.net/forum?id=rJ4vlh0qtm
['Xiangyu Kong', 'Jing Li', 'Bo Xin', 'Yizhou Wang'] ['gmail.com', 'pku.edu.cn']
228 https://openreview.net/forum?id=ryxHii09KQ
['Guy Hacohen', 'Daphna Weinshall'] ['mail.huji.ac.il', 'cs.huji.ac.il']
229 https://openreview.net/forum?id=HJeRkh05Km
['Wei Yang', 'Xiaolong Wang', 'Ali Farhadi', 'Abhinav Gupta', 'Roozbeh Mottaghi'] ['cs.washington.edu', 'cs.cmu.edu', 'ee.cuhk.edu.hk', 'allenai.org']
230 https://openreview.net/forum?id=BkMXkhA5Fm
['Pavel Solovev', 'Vladimir Aliev', 'Pavel Ostyakov', 'Gleb Sterkin', 'Elizaveta Logacheva', 'Stepan Troeshestov', 'Roman Suvorov', 'Anton Mashikhin', 'Oleg Khomenko', 'Sergey I. Nikolenko'] ['gmail.com'

...

['Alexander Tong', 'David van Dijk', 'Jay Stanley', 'Guy Wolf', 'Smita Krishnaswamy'] ['yale.edu']
1348 https://openreview.net/forum?id=r1g4E3C9t7
['Zhuolin Yang', 'Bo Li', 'Pin-Yu Chen', 'Dawn Song'] ['ibm.com', 'gmail.com', 'sjtu.edu.cn']
1349 https://openreview.net/forum?id=rJl8viCqKQ
['Alon Brutzkus', 'Oren Elisha', 'Ran Gilad-Bachrach'] ['gmail.com', 'microsoft.com']
1350 https://openreview.net/forum?id=rygnfn0qF7
['Ming-Wei Chang', 'Kristina Toutanova', 'Kenton Lee', 'Jacob Devlin'] ['google.com']
1351 https://openreview.net/forum?id=HygQBn0cYm
['Mikael Henaff', 'Alfredo Canziani', 'Yann LeCun'] ['cs.nyu.edu', 'nyu.edu']
1352 https://openreview.net/forum?id=BJl4f2A5tQ
['Kamyar Azizzadenesheli', 'Brandon Yang', 'Weitang Liu', 'Emma Brunskill', 'Zachary Lipton', 'Animashree Anandkumar'] ['stanford.edu', 'cs.stanford.edu', 'caltech.edu', 'cmu.edu', 'ucdavis.edu', 'uci.edu']
1353 https://openreview.net/forum?id=S1MB-3RcF7
['Isabela Albuquerque', 'João Monteiro', 'Thang Doan', 'Breand

['Rohin Shah', 'Dmitrii Krasheninnikov', 'Jordan Alexander', 'Pieter Abbeel', 'Anca Dragan'] ['cs.berkeley.edu', 'student.uva.nl', 'berkeley.edu', 'stanford.edu']
1401 https://openreview.net/forum?id=SkgE8sRcK7
['Bin Zhou', 'Jiashi Feng'] ['u.nus.edu']
1402 https://openreview.net/forum?id=HyzdRiR9Y7
['Mostafa Dehghani', 'Stephan Gouws', 'Oriol Vinyals', 'Jakob Uszkoreit', 'Lukasz Kaiser'] ['uva.nl', 'google.com']
1403 https://openreview.net/forum?id=HyeFAsRctQ
['Chongli Qin', 'Krishnamurthy (Dj) Dvijotham', "Brendan O'Donoghue", 'Rudy Bunel', 'Robert Stanforth', 'Sven Gowal', 'Jonathan Uesato', 'Grzegorz Swirszcz', 'Pushmeet Kohli'] ['google.com']
1404 https://openreview.net/forum?id=SkeJ6iR9Km
['Francesco Tonolini', 'Bjorn Sand Jensen', 'Roderick Murray-Smith'] ['student.gla.ac.uk', 'glasgow.ac.uk']
1405 https://openreview.net/forum?id=SyGjQ30qFX
['Yau-Shian Wang', 'Yun-Nung Chen', 'Hung-Yi Lee'] ['gmail.com', 'ieee.org']
1406 https://openreview.net/forum?id=SJl2ps0qKQ
['Haihong Yang'

['Yujun Li', 'Chengzhuo Ni', 'Guangzeng Xie', 'Wenhao Yang', 'Shuchang Zhou', 'Zhihua Zhang'] ['math.pku.edu.cn', 'gmail.com', 'pku.edu.cn', 'megvii.com']
1452 https://openreview.net/forum?id=HJgHwi0ctm
['Masoud Faraki', 'Mahsa Baktashmotlagh', 'Tom Drummond', 'Mathieu Salzmann'] ['qut.edu.au', 'monash.edu', 'epfl.ch']
1453 https://openreview.net/forum?id=HJx7l309Fm
['Shariq Iqbal', 'Fei Sha'] ['gmail.com']
1454 https://openreview.net/forum?id=HkfYOoCcYX
['Daehyun Ahn', 'Dongsoo Lee', 'Taesu Kim', 'Jae-Joon Kim'] ['gmail.com', 'postech.ac.kr']
1455 https://openreview.net/forum?id=SJMO2iCct7
['Daniel Hernandez Diaz', 'Antonio Khalil Moretti', 'Ziqiang Wei', 'Shreya Saxena', 'John Cunningham', 'Liam Paninski'] ['columbia.edu', 'gmail.com', 'cs.columbia.edu', 'janelia.hhmi.org']
1456 https://openreview.net/forum?id=H1lGHsA9KX
['Seong Jin Cho', 'Sunghun Kang', 'Chang D. Yoo'] ['kaist.ac.kr']
1457 https://openreview.net/forum?id=S1xNEhR9KX
['Gavin Weiguang Ding', 'Kry Yik Chau Lui', 'Xiaome

['Rong Ge', 'Rohith Kuditipudi', 'Zhize Li', 'Xiang Wang'] ['mails.tsinghua.edu.cn', 'cs.duke.edu', 'duke.edu']
1507 https://openreview.net/forum?id=Hyg1Ls0cKQ
['Jin-Young Kim', 'Sung-Bae Cho'] ['yonsei.ac.kr']
1508 https://openreview.net/forum?id=SyVhg20cK7
['David Earl Hostallero', 'Daewoo Kim', 'Kyunghwan Son', 'Yung Yi'] ['kaist.ac.kr', 'gmail.com', 'lanada.kaist.ac.kr', 'kaist.edu']
1509 https://openreview.net/forum?id=BJeWOi09FQ
['Cosmin I. Bercea', 'Olivier Pauly', 'Andreas K. Maier', 'Florin C. Ghesu'] ['gmail.com', 'siemens-healthineers.com', 'fau.de']
1510 https://openreview.net/forum?id=BkG8sjR5Km
['Siqi Liu', 'Guy Lever', 'Josh Merel', 'Saran Tunyasuvunakool', 'Nicolas Heess', 'Thore Graepel'] ['google.com']
1511 https://openreview.net/forum?id=rJe-LiA5YX
['Mitsuharu Takeori', 'Kenta Nakamura'] ['jp.nssol.nssmc.com']
1512 https://openreview.net/forum?id=rJeQYjRqYX
['Yuxian Qiu', 'Jingwen Leng', 'Yuhao Zhu', 'Quan Chen', 'Chao Li', 'Minyi Guo'] ['rochester.edu', 'sjtu.edu.cn

['Jiacang Ho', 'Dae-Ki Kang'] ['dongseo.ac.kr', 'hotmail.com']
1564 https://openreview.net/forum?id=BkxkH30cFm
['Guangxiang Zhu', 'Jianhao Wang', 'ZhiZhou Ren', 'Chongjie Zhang'] ['mails.tsinghua.edu.cn', 'tsinghua.edu.cn', 'outlook.com']
1565 https://openreview.net/forum?id=ByGuynAct7
['Andrei Atanov', 'Arsenii Ashukha', 'Kirill Struminsky', 'Dmitriy Vetrov', 'Max Welling'] ['uva.nl', 'gmail.com', 'yandex.ru']
1566 https://openreview.net/forum?id=Byf5-30qFX
['Meng Fang', 'Cheng Zhou', 'Bei Shi', 'Boqing Gong', 'Jia Xu', 'Tong Zhang'] ['cs.wisc.edu', 'gmail.com', 'tongzhang-ml.org', 'outlook.com']
1567 https://openreview.net/forum?id=Syl8Sn0cK7
['Xujie Si', 'Yuan Yang', 'Hanjun Dai', 'Mayur Naik', 'Le Song'] ['cis.upenn.edu', 'cc.gatech.edu', 'gatech.edu']
1568 https://openreview.net/forum?id=B1MB5oRqtQ
['Dmitry Kangin', 'Nicolas Pugeault'] ['exeter.ac.uk']
1569 https://openreview.net/forum?id=S1fcnoR9K7
['Léonard Blier', 'Pierre Wolinski', 'Yann Ollivier'] ['u-psud.fr', 'fb.com']
1570

In [78]:
print(len(papers))

1579


In [79]:
import copy
scraped_papers = copy.deepcopy(papers)
print(len(papers), idx)

1579 1579


In [80]:


papers = copy.deepcopy(scraped_papers)
for paper in list(papers.values()):
    if 'comments' not in paper:
        continue
    for idx in range(len(paper['comments'])):
        comment = paper['comments'][idx]
        comment = {k.strip(': '): v for k,v in comment.items()}
        if 'Rating' in comment:
            comment['Rating'] = re.search(r'^\d+', comment['Rating']).group()
        if 'Confidence' in comment:
            comment['Confidence'] = re.search(r'^\d+', comment['Confidence']).group()
        else:
            comment['Confidence'] = '4'
        paper['comments'][idx] = comment


In [81]:
def generate_top_papers(papers):
    overall_ratings = {}
    for paper in list(papers.values()):
        real_ratings = []
        ratings = []
        confidence = []
        if 'comments' not in paper:
            continue
        for comment in paper['comments']:
            if 'Rating' in comment:
                real_ratings.append(int(comment['Rating']))
                ratings.append(int(comment['Rating'])*int(comment['Confidence']))
                confidence.append(int(comment['Confidence']))

        if len(ratings) == 0:
            continue
    #     print(paper['title'])
        overall_ratings[paper['title']] = (sum(ratings)/(len(ratings)*(sum(confidence)/len(confidence))), real_ratings, confidence)

    print(len(overall_ratings))
    all_ratings = sorted(list(overall_ratings.items()), key=lambda x:x[1][0], reverse=True)
    for name, rating in all_ratings:
        print(f'{"%.2f" % rating[0]},{name},{tuple(rating[1])},{tuple(rating[2])}')
generate_top_papers(papers)

1558
8.91,GENERATING HIGH FIDELITY IMAGES WITH SUBSCALE PIXEL NETWORKS AND MULTIDIMENSIONAL UPSCALING,(7, 10, 9),(3, 5, 3)
8.79,ALISTA: Analytic Weights Are As Good As Learned Weights in LISTA,(10, 7, 9),(5, 4, 5)
8.50,Benchmarking Neural Network Robustness to Common Corruptions and Perturbations,(7, 9, 9),(3, 5, 4)
8.33,On Random Deep Weight-Tied Autoencoders: Exact Asymptotic Analysis, Phase Transitions, and Implications to Training,(9, 8, 8),(4, 4, 4)
8.20,Variational Discriminator Bottleneck: Improving Imitation Learning, Inverse RL, and GANs by Constraining Information Flow,(6, 10, 8),(3, 4, 3)
8.09,Large Scale GAN Training for High Fidelity Natural Image Synthesis,(8, 7, 9),(4, 3, 4)
8.09,Ordered Neurons: Integrating Tree Structures into Recurrent Neural Networks,(9, 7, 8),(4, 3, 4)
8.08,Slimmable Neural Networks,(8, 9, 7),(4, 5, 4)
8.08,An Empirical Study of Example Forgetting during Deep Neural Network Learning,(9, 8, 7),(5, 4, 4)
8.00,BA-Net: Dense Bundle Adjustment Networks,(

In [82]:
final_json = []
review_scores = []
confidence_scores = []
for url, paper in papers.items():
    result = {}
    result['url'] = url
    real_ratings = []
    ratings = []
    confidence = []
    if 'comments' in paper:
        for comment in paper['comments']:
            if 'Rating' in comment and 'Confidence' in comment:
                real_ratings.append(int(comment['Rating']))
                ratings.append(int(comment['Rating'])*int(comment['Confidence']))
                review_scores.append(int(comment['Rating']))
                confidence_scores.append(int(comment['Confidence']))
                confidence.append(int(comment['Confidence']))
            elif 'Decision' in comment:
                result['decision'] = comment['Decision']
            elif 'Recommendation' in comment:
                result['decision'] = comment['Recommendation']
            
        result['comments'] = paper['comments']
    result['ratings'] = real_ratings
    if len(ratings) != 0:
        result['rating'] = "%.2f" % (sum(ratings)/(len(ratings)*(sum(confidence)/len(confidence))))
    else:
        result['rating'] = ' N/A'
    result['confidences'] = confidence
    result['abstract'] = paper['abstract']
    result['title'] = paper['title']
    result['authors'] = paper['authors']
    result['emails'] = paper['emails']
    final_json.append(result)

sorted_final = sorted(final_json, key=lambda x: x['rating'], reverse=True)
final_json = []
for idx, i in enumerate(sorted_final):
    i['rank'] = idx+1
    final_json.append(i)

f = open('data.json', 'w')
print(f.write(json.dumps(final_json, indent=2)))
f.close()

34981141


In [83]:
import numpy as np

f = open('data.json', 'r')
final_json = json.loads(f.read())
f.close()

ratings = [float(i['rating']) for i in final_json if i['rating'] != ' N/A']
ratings = sorted(ratings)
print(sum(ratings)/len(ratings))
print(ratings[len(ratings)*3//4])
# review_dict = {k:0 for k in range(1, 11)}
# for i in review_scores:
#     review_dict[i] += 1
# print(sum(review_scores)/len(review_scores))
# print(review_dict)
# confidence_dict = {k:0 for k in range(1, 6)}
# for i in confidence_scores:
#     confidence_dict[i] += 1
# print(confidence_dict)
def variance(arr, conf):
    if len(arr) == 0:
        return 0
    return np.var(arr)

#     mu = sum(arr)/len(arr)
#     var = 0
#     for idx in range(len(arr)):
#         var += (arr[idx] - mu)**2
#     )
#     return var/len(arr)

variance = [(i['title'], variance(i['ratings'], i['confidences'])) for i in final_json]

variance = sorted(variance, key=lambda x:x[1], reverse=True)
variance_dict = {k:v for k,v in variance}
for title, v in variance[:10]:
    print(f'{"%.2f" % v}: {title}')
    
entropies = [(i['title'], stats.entropy(i['ratings'])/len(i['ratings'])) for i in final_json]
entropies = sorted(entropies, key=lambda x:x[1])
print(entropies[:10])
for idx, i in enumerate(final_json):
    i['variance'] = float("%.2f" % variance_dict[i['title']]**(1/2))
    final_json[idx] = i
    
f = open('data.json', 'w')
print(f.write(json.dumps(final_json, indent=2)))
f.close()
# print(final_json)

5.300198083067093
6.29
6.89:  Large-Scale Visual Speech Recognition
6.22: Caveats for information bottleneck in deterministic scenarios
6.22: Why Do Neural Response Generation Models Prefer Universal Replies?
6.22: Optimization on Multiple Manifolds
5.56: Biologically-Plausible Learning Algorithms Can Scale to Large Datasets
5.56: PPO-CMA: Proximal Policy Optimization with Covariance Matrix Adaptation
5.56: Learning to Adapt in Dynamic, Real-World Environments through Meta-Reinforcement Learning
5.25: Exploration by random network distillation
4.67: Invariant and Equivariant Graph Networks
4.67: Per-Tensor Fixed-Point Quantization of the Back-Propagation Algorithm
[('A Main/Subsidiary Network Framework for Simplifying Binary Neural Networks', 0.0), ('Advanced Neuroevolution: A gradient-free algorithm to train Deep Neural Networks', 0.26543721339127097), ('Optimization on Multiple Manifolds', 0.2866557603451683), ('Why Do Neural Response Generation Models Prefer Universal Replies?', 0.2

/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in double_scalars


35015641


1331551
